In [1]:
import drjit as dr
import mitsuba as mi

mi.set_variant('llvm_ad_rgb')
import plugins


[mitsuba] Warning: Couldn't import the ipywidgets package. Installing this package is required for the system to properly log messages and print in Jupyter notebooks!


In [2]:
xml = """<float name="roughness" value="0.05"/>
         <rgb name="albedo" value="1.0"/>
         <rgb name="specular" value="0.0"/>"""
sample_func, pdf_func = mi.chi2.BSDFAdapter("rednermat", xml)

chi2 = mi.chi2.ChiSquareTest(
    domain=mi.chi2.SphericalDomain(),
    sample_func=sample_func,
    pdf_func=pdf_func,
    sample_dim=3,
    ires=16,
    res=201
)

assert chi2.run()

RuntimeError: ​[xml.cpp:1115] Error while loading "<string>" (near line 1, col 1): could not instantiate bsdf plugin of type "rednermat": TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
[xml.cpp:1115]     1. mitsuba.llvm_ad_rgb.Texture(props: mitsuba.llvm_ad_rgb.Properties)
[xml.cpp:1115] 
[xml.cpp:1115] Invoked with: 0.1
[xml.cpp:1115] 
[xml.cpp:1115] At:
[xml.cpp:1115]   /home/beltegeuse/project/NextFace/plugins/bsdfs/rednermat.py(14): __init__
[xml.cpp:1115]   /home/beltegeuse/project/NextFace/plugins/__init__.py(6): <lambda>
[xml.cpp:1115]   /home/beltegeuse/project/NextFace/mitsuba/python/chi2.py(504): instantiate
[xml.cpp:1115]   /home/beltegeuse/project/NextFace/mitsuba/python/chi2.py(508): sample_functor
[xml.cpp:1115]   /home/beltegeuse/project/NextFace/mitsuba/python/chi2.py(125): tabulate_histogram
[xml.cpp:1115]   /home/beltegeuse/project/NextFace/mitsuba/python/chi2.py(261): run
[xml.cpp:1115]   /tmp/ipykernel_17451/38